# Reshaping the CDC data and calculate significances

The data exports from the CDC Wonder database include a good deal of cruft, and have different sets of columns, depending on what you export. This notebook cleans and standardizes the CDC data we downloaded. Then, we use the CDC's own approach, [outlined here](https://www.cdc.gov/mmwr/volumes/67/wr/mm6712a1.htm), to determine whether the year-over-year changes in death rates are statistically signficant.

In [1]:
import pandas as pd
import numpy as np

This function loads the data, flags suppressed values, and removes the "Notes" rows:

In [2]:
def parse_mcd_file(path):
    raw = pd.read_csv(
        path,
        delimiter = "\t",
        dtype = {
            "State Code": str
        }
    )
    

    df = (
        raw

        # Filter out the "Notes" rows
        .loc[lambda x: x["Notes"].isnull()]
        
        # Drop superfluous columns
        .drop(
            columns = [ "Notes", "Year Code", "Crude Rate" ],
        )
        
        # Flag rows with suppressed values
        .assign(
            deaths_flag = lambda x: (
                x["Deaths"].astype(str)
                .replace(r"^[\d\.]+$", "", regex = True)
            ),
            adj_rate_flag = lambda x: (
                x["Age Adjusted Rate"].astype(str)
                .replace(r"^[\d\.]+$", "", regex = True)
            ),
        )
        
        # Remove non-population-specified rows,
        # which occurs for Hispanic origin "Not Stated"
        .loc[lambda x: x["Population"].astype(str) != "Not Applicable"]
    )

    flag_strings = [
        "Suppressed",
        "Unreliable",
        "Not Applicable",
        "Missing"
    ]
    
    
    # Nullify flags
    for c in df.columns:
        if c == "Deaths" or "Rate" in c:            
            df.loc[lambda x: x[c].isin(flag_strings), c] = pd.np.nan
            df[c] = df[c].astype(float)

    df["Year"] = df["Year"].astype(int)
    df["Population"] = df["Population"].astype(int)

    return df

This function takes the data cleaned by the function above, and reshapes it into something more standardized, so that we generally have the same set of columns across our datasets:

In [3]:
def reshape_mcd_data(df, id_cols, drug_abbrev):
    return (
        df
        
        # Remove columns we don"t use
        .drop(
            columns = [
                "Race Code",
                "Hispanic Origin",
                "Hispanic Origin Code"
            ],
            errors = "ignore"
        )
        
        # Simplify the other column names
        .rename(columns = {
            "Year": "year",
            "Population": "population",
            "Deaths": "deaths",
            "Age Adjusted Rate": "adj_rate",
            "Age Adjusted Rate Upper 95% Confidence Interval": "upper_int",
            "Age Adjusted Rate Lower 95% Confidence Interval": "lower_int",
        })
        
        # Melt the columns so that we can create a 
        # sub-variable for each of the two years.
        .melt(
            id_vars = [ "year" ] + id_cols
        )
        .assign(
            drug = drug_abbrev
        )
        .assign(
            variable = lambda x: (
                x["variable"] + "_" + 
                x["year"].astype(str).str.slice(-2)
            )
        )
        
         # Un-melt the dataframe
        .set_index(id_cols + [ "drug", "variable" ])
        ["value"]
        .unstack()
        .reset_index()
    )

### Reshape race/ethnicity data

In [4]:
all_drugs_race_non_hispanic = (
    parse_mcd_file("../data/mcd/race/allDrugs_15_16_by_race_and_ethnicity.txt")
    .loc[lambda x: x["Hispanic Origin"] == "Not Hispanic or Latino"]
    .pipe(reshape_mcd_data, [ "Race" ], "all")
)

all_drugs_race_non_hispanic


variable,Race,drug,adj_rate_15,adj_rate_16,adj_rate_flag_15,adj_rate_flag_16,deaths_15,deaths_16,deaths_flag_15,deaths_flag_16,lower_int_15,lower_int_16,population_15,population_16,upper_int_15,upper_int_16
0,American Indian or Alaska Native,all,21.23,24.24,,,553,638,,,19.42,22.32,2689706,2711067,23.03,26.16
1,Asian or Pacific Islander,all,2.69,3.1,,,548,644,,,2.46,2.86,19116557,19479730,2.91,3.35
2,Black or African American,all,12.18,17.08,,,5070,7220,,,11.84,16.68,41777483,42141669,12.52,17.48
3,White,all,21.07,25.3,,,41720,49457,,,20.86,25.07,201242281,201324760,21.28,25.53


In [5]:
coc_fen_race_non_hispanic = (
    parse_mcd_file("../data/mcd/race/cocaine_fentanyl_15_16_by_race_and_ethnicity.txt")
    .loc[lambda x: x["Hispanic Origin"] == "Not Hispanic or Latino"]
    .pipe(reshape_mcd_data, [ "Race" ], "coc_fen")
)
coc_fen_race_non_hispanic

variable,Race,drug,adj_rate_15,adj_rate_16,adj_rate_flag_15,adj_rate_flag_16,deaths_15,deaths_16,deaths_flag_15,deaths_flag_16,lower_int_15,lower_int_16,population_15,population_16,upper_int_15,upper_int_16
0,American Indian or Alaska Native,coc_fen,NaN,NaN,Unreliable,Unreliable,5,15,,,0.07,0.32,2689706,2711067,0.48,0.94
1,Asian or Pacific Islander,coc_fen,NaN,0.1,Unreliable,,8,20,,,0.02,0.06,19116557,19479730,0.08,0.15
2,Black or African American,coc_fen,0.56,1.88,,,243,779,,,0.48,1.74,41777483,42141669,0.63,2.01
3,White,coc_fen,0.62,1.6,,,1151,2936,,,0.58,1.54,201242281,201324760,0.65,1.66


In [6]:
coc_fen_race_hispanic = (
    parse_mcd_file("../data/mcd/race/cocaine_fentanyl_15_16_by_ethnicity.txt")
    .rename(columns = { "Hispanic Origin": "Race" })
    .loc[lambda x: x["Race"] == "Hispanic or Latino"]
    .pipe(reshape_mcd_data, [ "Race" ], "coc_fen")
)
coc_fen_race_hispanic

variable,Race,drug,adj_rate_15,adj_rate_16,adj_rate_flag_15,adj_rate_flag_16,deaths_15,deaths_16,deaths_flag_15,deaths_flag_16,lower_int_15,lower_int_16,population_15,population_16,upper_int_15,upper_int_16
0,Hispanic or Latino,coc_fen,0.23,0.69,,,124,403,,,0.19,0.63,56592793,57470287,0.28,0.76


In [7]:
all_drugs_race_hispanic = (
    parse_mcd_file("../data/mcd/race/allDrugs_15_16_by_ethnicity.txt")
    .rename(columns = { "Hispanic Origin": "Race" })
    .loc[lambda x: x["Race"] == "Hispanic or Latino"]
    .pipe(reshape_mcd_data, [ "Race" ], "all")
)
all_drugs_race_hispanic

variable,Race,drug,adj_rate_15,adj_rate_16,adj_rate_flag_15,adj_rate_flag_16,deaths_15,deaths_16,deaths_flag_15,deaths_flag_16,lower_int_15,lower_int_16,population_15,population_16,upper_int_15,upper_int_16
0,Hispanic or Latino,all,7.65,9.48,,,4117,5230,,,7.41,9.22,56592793,57470287,7.89,9.75


In [8]:
rates_by_race = pd.concat([
    all_drugs_race_hispanic,
    all_drugs_race_non_hispanic,
    coc_fen_race_hispanic,
    coc_fen_race_non_hispanic
])

### Reshape state data

In [9]:
coc_fen_state = (
    parse_mcd_file("../data/mcd/state/cocaine_fentanyl_15_16_by_state.txt")
    .pipe(reshape_mcd_data, [ "State", "State Code" ], "coc_fen")
)
coc_fen_state.head()

variable,State,State Code,drug,adj_rate_15,adj_rate_16,adj_rate_flag_15,adj_rate_flag_16,deaths_15,deaths_16,deaths_flag_15,deaths_flag_16,lower_int_15,lower_int_16,population_15,population_16,upper_int_15,upper_int_16
0,Alabama,01,coc_fen,NaN,0.75,Suppressed,,NaN,31,Suppressed,,NaN,0.51,4858979,4863300,NaN,1.07
1,Alaska,02,coc_fen,NaN,NaN,Suppressed,Suppressed,NaN,NaN,Suppressed,Suppressed,NaN,NaN,738432,741894,NaN,NaN
2,Arizona,04,coc_fen,NaN,NaN,Suppressed,Unreliable,NaN,13,Suppressed,,NaN,0.1,6828065,6931071,NaN,0.31
3,Arkansas,05,coc_fen,NaN,NaN,Suppressed,Suppressed,NaN,NaN,Suppressed,Suppressed,NaN,NaN,2978204,2988248,NaN,NaN
4,California,06,coc_fen,NaN,0.11,Unreliable,,15,45,,,0.02,0.08,39144818,39250017,0.07,0.14


In [10]:
coc_state = (
    parse_mcd_file("../data/mcd/state/cocaine_15_16_by_state.txt")
    .pipe(reshape_mcd_data, [ "State", "State Code" ], "coc")
)
coc_state.head()

variable,State,State Code,drug,adj_rate_15,adj_rate_16,adj_rate_flag_15,adj_rate_flag_16,deaths_15,deaths_16,deaths_flag_15,deaths_flag_16,lower_int_15,lower_int_16,population_15,population_16,upper_int_15,upper_int_16
0,Alabama,01,coc,0.83,1.78,,,38,82,,,0.58,1.41,4858979,4863300,1.14,2.22
1,Alaska,02,coc,NaN,NaN,Suppressed,Unreliable,NaN,15,Suppressed,,NaN,0.99,738432,741894,NaN,3.04
2,Arizona,04,coc,0.94,1.22,,,62,82,,,0.72,0.96,6828065,6931071,1.22,1.52
3,Arkansas,05,coc,NaN,NaN,Unreliable,Unreliable,14,10,,,0.29,0.17,2978204,2988248,0.9,0.64
4,California,06,coc,0.72,0.86,,,296,366,,,0.64,0.78,39144818,39250017,0.8,0.95


In [11]:
rates_by_state = pd.concat([
    coc_state,
    coc_fen_state
])

## Calculate significance

Calculate z-tests for year-over-year proportions. If either population value is below 100, we instead do a simple comparison of the provided confidence intervals for each rate. The method is used in [this CDC report](https://www.cdc.gov/mmwr/volumes/67/wr/mm6712a1.htm) and discussed at length in this [National Vital Statistics Report](https://www.cdc.gov/nchs/data/nvsr/nvsr65/nvsr65_04.pdf).

In [12]:
def calc_zscore(series):
    # proportions
    p1 = series["adj_rate_15"] / 100000
    p2 = series["adj_rate_16"] / 100000
    
    # sample sizes
    pop1 = series["population_15"]
    pop2 = series["population_16"]

    # overall proportion
    p = ((p1 * pop1) + (p2 * pop2)) / (pop1 + pop2)

    denom = p * (1 - p) * ((1 / pop1) + (1 / pop2))

    z = (np.abs(p1 - p2) - 0) / np.sqrt(denom)
    
    return z

In [13]:
def test_nonoverlapping_intervals(x):
    overlap = (
        min(x["upper_int_15"], x["upper_int_16"]) - 
        max(x["lower_int_15"], x["lower_int_16"])
    )
    return overlap < 0

In [14]:
def add_significance_cols(df):
    df = df.copy()
    
    df["gt_100_deaths"] = (
        df[["deaths_15", "deaths_16"]]
        .fillna(0)
        .min(axis = 1) > 100
    )
    
    df["nonoverlapping_intervals"] = (
        df
        .apply(test_nonoverlapping_intervals, axis = 1)
    )
    
    df["zscore"] = df.apply(calc_zscore, axis = 1)
    
    
    
    df["sig"] = (
        (df["gt_100_deaths"] & (df["zscore"] > 1.96)) |
        (~df["gt_100_deaths"] & (df["nonoverlapping_intervals"] == True)) 
    )
    
    # Distinguish between non-significant annual changes
    # and annual changes that are missing data.
    df.loc[lambda x: (
        (x["sig"] == False) &
        (x["lower_int_16"].isnull() | x["lower_int_15"].isnull())
    ), "sig"] = None
    
    return df

In [15]:
rates_by_race.head(1)

variable,Race,drug,adj_rate_15,adj_rate_16,adj_rate_flag_15,adj_rate_flag_16,deaths_15,deaths_16,deaths_flag_15,deaths_flag_16,lower_int_15,lower_int_16,population_15,population_16,upper_int_15,upper_int_16
0,Hispanic or Latino,all,7.65,9.48,,,4117,5230,,,7.41,9.22,56592793,57470287,7.89,9.75


In [16]:
rates_by_race = add_significance_cols(rates_by_race)
rates_by_race.head().T

,0,0,1,2,3
variable,,,,,
Race,Hispanic or Latino,American Indian or Alaska Native,Asian or Pacific Islander,Black or African American,White
drug,all,all,all,all,all
adj_rate_15,7.65,21.23,2.69,12.18,21.07
adj_rate_16,9.48,24.24,3.1,17.08,25.3
adj_rate_flag_15,,,,,
adj_rate_flag_16,,,,,
deaths_15,4117,553,548,5070,41720
deaths_16,5230,638,644,7220,49457
deaths_flag_15,,,,,


The quality of data reported by medical examiners varies by state. The CDC has noted that some states have a tendency to report drug poisoning deaths without listing a specific drug. Because of this, researchers have identified states with [high-quality reporting](https://www.cdc.gov/mmwr/volumes/67/wr/mm6712a1.htm). Here, we incorporate this research to identify states that don't meet that threshold.

In [17]:
CDC_QUALITY_STATES = [
    "Alaska",
    "Connecticut",
    "District of Columbia",
    "Georgia",
    "Illinois",
    "Iowa",
    "Maine",
    "Maryland",
    "Massachusetts",
    "Nevada",
    "New Hampshire",
    "New Mexico",
    "New York",
    "North Carolina",
    "Ohio",
    "Oklahoma",
    "Oregon",
    "Rhode Island",
    "South Carolina",
    "Tennessee",
    "Utah",
    "Vermont",
    "Virginia",
    "Washington",
    "West Virginia",
    "Arizona",
    "Colorado",
    "Hawaii",
    "Minnesota",
    "Missouri",
    "Texas",
    "Wisconsin"
]

In [18]:
rates_by_state = (
    add_significance_cols(rates_by_state)
    .assign(
        quality_reporting = lambda x: x["State"].isin(CDC_QUALITY_STATES)
    )
)
rates_by_state.head().T

,0,1,2,3,4
variable,,,,,
State,Alabama,Alaska,Arizona,Arkansas,California
State Code,01,02,04,05,06
drug,coc,coc,coc,coc,coc
adj_rate_15,0.83,NaN,0.94,NaN,0.72
adj_rate_16,1.78,NaN,1.22,NaN,0.86
adj_rate_flag_15,,Suppressed,,Unreliable,
adj_rate_flag_16,,Unreliable,,Unreliable,
deaths_15,38,NaN,62,14,296
deaths_16,82,15,82,10,366


## Save results

In [19]:
rates_by_race.to_csv(
    "../output/rates-by-race.csv",
    index = False
)

In [20]:
rates_by_state.to_csv(
    "../output/rates-by-state.csv",
    index = False    
)

## States with statistically-significant cocaine+fentanyl annual change

In [21]:
(
    rates_by_state
    .loc[lambda x: x["drug"] == "coc_fen"]
    [[
        "State",
        "deaths_15",
        "deaths_16",
        "zscore",
        "nonoverlapping_intervals",
        "sig",
        "quality_reporting",
    ]]
    .loc[lambda x: x["sig"] == True]
)

variable,State,deaths_15,deaths_16,zscore,nonoverlapping_intervals,sig,quality_reporting
4,California,15,45,NaN,True,1.0,False
6,Connecticut,40,128,7.238872,True,1.0,True
9,Florida,148,480,13.828513,True,1.0,False
13,Illinois,36,163,9.559315,True,1.0,True
19,Maine,15,39,NaN,True,1.0,True
20,Maryland,42,169,8.667793,True,1.0,True
21,Massachusetts,220,399,7.216405,True,1.0,True
22,Michigan,112,260,8.148628,True,1.0,False
25,Missouri,10,34,NaN,True,1.0,True
30,New Jersey,43,170,9.045405,True,1.0,False


### States that acheived significance via confidence intervals

In [22]:
rates_by_state.loc[
    lambda x: (x["zscore"].isnull()) & (x["sig"] == True)
][[
    "State",
    "drug",
    "zscore",
    "sig",
    "adj_rate_15",
    "adj_rate_16",
    "lower_int_16",
    "upper_int_15",
    "lower_int_15",
    "upper_int_16",
]]


variable,State,drug,zscore,sig,adj_rate_15,adj_rate_16,lower_int_16,upper_int_15,lower_int_15,upper_int_16
4,California,coc_fen,NaN,1.0,NaN,0.11,0.08,0.07,0.02,0.14
19,Maine,coc_fen,NaN,1.0,NaN,3.32,2.34,2.25,0.76,4.57
25,Missouri,coc_fen,NaN,1.0,NaN,0.61,0.42,0.31,0.07,0.86
49,Wisconsin,coc_fen,NaN,1.0,NaN,0.73,0.52,0.43,0.14,1


### States that were missing either or both confidence intervals

In [23]:
rates_by_state.loc[
    lambda x: (x["sig"].isnull())
][[
    "State",
    "drug",
    "zscore",
    "sig",
    "adj_rate_15",
    "adj_rate_16",
    "lower_int_16",
    "upper_int_16",
    "lower_int_15",
    "upper_int_15",
]]


variable,State,drug,zscore,sig,adj_rate_15,adj_rate_16,lower_int_16,upper_int_16,lower_int_15,upper_int_15
1,Alaska,coc,NaN,NaN,NaN,NaN,0.99,3.04,NaN,NaN
11,Hawaii,coc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,Idaho,coc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,Montana,coc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,Nebraska,coc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,North Dakota,coc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,South Dakota,coc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,Wyoming,coc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Alabama,coc_fen,NaN,NaN,NaN,0.75,0.51,1.07,NaN,NaN
1,Alaska,coc_fen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---

---

---